## 1. Tool Creation

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
# tool creation

@tool # decorator
def multiply(a: int, b:int) -> int:
    """Given two numbers a and b, this tool returns there product"""
    return a * b

In [3]:
print(multiply.invoke({"a":3, "b":5}))

15


In [12]:
multiply.name 

'multiply'

In [14]:
multiply.description

'Given two numbers a and b, this tool returns there product'

In [15]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

## 2.Tool Binding

In [34]:
# Tool Binding! 
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [35]:
llm_with_tools = llm.bind_tools([multiply])      # It's a list can add more tools 
# tool binding ^

## 3. Tool Calling

In [36]:
llm_with_tools.invoke("Hell, yeah how are you?")

AIMessage(content="I'm doing well, thank you for asking!  How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--22a0fd3d-100e-4bab-b0d3-91614eb31c37-0', usage_metadata={'input_tokens': 27, 'output_tokens': 18, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})

In [77]:
query = HumanMessage('can you multiply 3 with 1002')

In [78]:
messages = [query]

In [79]:
messages

[HumanMessage(content='can you multiply 3 with 1002', additional_kwargs={}, response_metadata={})]

In [80]:
llm_with_tools.invoke("Can you multiply 3 wtih 5?").tool_calls[0]      # fetching tool calls cause this is an attribute!

{'name': 'multiply',
 'args': {'a': 3.0, 'b': 5.0},
 'id': '387dbe34-2f96-44ff-af3f-74721150543a',
 'type': 'tool_call'}

In [ ]:
# llm advices which tools would be a great option for query, llm doesn't call a tool!
# It's either called by Langchain or User(dev)

# ⚠️Important!!!
    - llm advices which tools would be a great option for query, llm doesn't call a tool!
    - It's either called by Langchain or User(dev)

In [82]:
result = llm_with_tools.invoke(messages)

In [83]:
messages.append(result)

In [84]:
messages

[HumanMessage(content='can you multiply 3 with 1002', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1002.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--e9ba7c88-e787-4c84-abe8-192857872e55-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1002.0}, 'id': 'f2d54c81-12d1-41cf-9aab-d53d134c6829', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [85]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3.0, 'b': 1002.0},
 'id': 'f2d54c81-12d1-41cf-9aab-d53d134c6829',
 'type': 'tool_call'}

In [86]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='3006', name='multiply', tool_call_id='f2d54c81-12d1-41cf-9aab-d53d134c6829')

In [87]:
tool_result = multiply.invoke(result.tool_calls[0]) # tool execution  also type of message

In [88]:
messages.append(tool_result) 
# humanMessage, AIMessage and Now ToolMessage

In [89]:
messages

[HumanMessage(content='can you multiply 3 with 1002', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1002.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--e9ba7c88-e787-4c84-abe8-192857872e55-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1002.0}, 'id': 'f2d54c81-12d1-41cf-9aab-d53d134c6829', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3006', name='multiply', tool_call_id='f2d54c81-12d1-41cf-9aab-d53d134c6829')]

    - above we made and stored an conversation history!

In [90]:
llm_with_tools.invoke(messages).content

'3 multiplied by 1002 is 3006.'